This was used to test some random stuff, you can skip reading it (or not). I found Jupyter notebook to be extremely convinient.
Anyway, this code is very badly organized.

In [66]:
import glob
import os
import sys

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

import carla
import numpy as np
import time
import random
from utility import *

client = carla.Client('localhost', 2000)
client.set_timeout(30.0)

world = client.get_world()

actor_list = []

In [75]:

# Get map and roads
map = world.get_map()
roads = map.get_topology()
spectator = world.get_spectator()
spawn_points = map.get_spawn_points()

# Get waypoints
distance = 1.0
waypoints = map.generate_waypoints(distance)

# Visualize
for waypoint in waypoints:
    world.debug.draw_string(waypoint.transform.location, '0', draw_shadow=False, 
                            color=carla.Color(255,0,0), life_time=100.0, 
                            persistent_lines=True)
    
time.sleep(50)


In [ ]:
for i, spawn_point in enumerate(spawn_points):
    location = spawn_point.location
    print(f"Spawn point {i+1}: (x={location.x}, y={location.y}, z={location.z})")

In [23]:
# Visualize lanes using Debug Lines

waypoint = random.choice(waypoints)
print(waypoint.transform.location)

for y in ys:
    location = carla.Location(x,y,z=0)
    world.debug.draw_string(location, '0', draw_shadow=False, 
                            color=carla.Color(200,200,0), life_time=50.0, 
                            persistent_lines=True)
    time.sleep(1)


Location(x=92.388222, y=119.840034, z=0.000000)


In [139]:
# Spawn some vehicles and waypoints
vehicle_startpoint = carla.Transform(carla.Location(x=-2.03, y= 9.56, z=0.3), 
                                            carla.Rotation(pitch=0.0, yaw=90.0, roll=0.0))
# vehicle_startpoint = carla.Transform(carla.Location(x=-2.03, y= 119.341, z=0.3), 
#                                             carla.Rotation(pitch=0.0, yaw=90.0, roll=0.0))
vehicle = spawn_vehicle(world, vehicle_startpoint)
actor_list.append(vehicle)



Wow! You're reading this! Appreciated.

In [113]:
# Get first waypoint
vehicle_location = vehicle.get_location()
vehicle_dimensions = get_vehicle_dimensions(vehicle)
forward_vector = vehicle.get_transform().get_forward_vector()

first_waypoint_location = vehicle_location + forward_vector * vehicle_dimensions[0]
first_waypoint = map.get_waypoint(first_waypoint_location)

# Visualize
world.debug.draw_point(first_waypoint.transform.location, size=0.5, color=carla.Color(255,0,0), life_time=10.0)
world.debug.draw_string(first_waypoint.transform.location, '0', draw_shadow=False, 
                        color=carla.Color(255,255,255), life_time=10.0, 
                        persistent_lines=True)


In [154]:
def get_waypoints(vehicle, distance=1.0, num_waypoints=10):
    waypoints = []
    left_lane_points = []
    right_lane_points = []

    reverse = False

    vehicle_location = vehicle.get_location()
    vehicle_dimensions = get_vehicle_dimensions(vehicle)
    forward_vector = vehicle.get_transform().get_forward_vector()

    first_waypoint_location = vehicle_location
    first_waypoint = map.get_waypoint(first_waypoint_location)
    waypoints.append(first_waypoint)
    left_lane_points.append(first_waypoint.get_left_lane())
    right_lane_points.append(first_waypoint.get_right_lane())

    if abs(first_waypoint.transform.rotation.yaw - vehicle.get_transform().rotation.yaw) > 45:
        reverse = True

    current_waypoint = first_waypoint

    for _ in range(num_waypoints):
        next_waypoint = current_waypoint.previous(distance)[0] if reverse else current_waypoint.next(distance)[0]

        if not next_waypoint:
            break

        waypoints.append(next_waypoint)
        current_waypoint = next_waypoint

    return waypoints

waypoints = get_waypoints(vehicle)
for i, waypoint in enumerate(waypoints):
    world.debug.draw_point(waypoint.transform.location, size=0.1, color=carla.Color(0, 255, 255), life_time=10.0)
    print(f"Waypoint {i + 1}: (x={waypoint.transform.location.x}, y={waypoint.transform.location.y}, z={waypoint.transform.location.z}). yaw {waypoint.transform.rotation.yaw}")

def get_lane_points(vehicle, waypoints):
    left_points = []
    right_points = []

    vehicle_width = get_vehicle_dimensions(vehicle)[1]/2

    for waypoint in waypoints:
        location = waypoint.transform.location

        left_vector = waypoint.transform.get_left_vector()
        right_vector = waypoint.transform.get_left_vector()

        left_points.append(location + left_vector*vehicle_width)
        right_points.append(location + right_vector*vehicle_width)

        




Waypoint 1: (x=-2.029846668243408, y=9.561538696289062, z=0.0). yaw 90.02207946777344
Waypoint 2: (x=-2.0302319526672363, y=10.561538696289062, z=0.0). yaw 90.02206420898438
Waypoint 3: (x=-2.0306169986724854, y=11.561538696289062, z=0.0). yaw 90.02206420898438
Waypoint 4: (x=-2.0310022830963135, y=12.561538696289062, z=0.0). yaw 90.02206420898438
Waypoint 5: (x=-2.0313875675201416, y=13.561538696289062, z=0.0). yaw 90.02206420898438
Waypoint 6: (x=-2.0317726135253906, y=14.561538696289062, z=0.0). yaw 90.02206420898438
Waypoint 7: (x=-2.0321578979492188, y=15.561538696289062, z=0.0). yaw 90.02206420898438
Waypoint 8: (x=-2.032543182373047, y=16.561538696289062, z=0.0). yaw 90.02206420898438
Waypoint 9: (x=-2.032928466796875, y=17.561538696289062, z=0.0). yaw 90.02206420898438
Waypoint 10: (x=-2.033313512802124, y=18.561538696289062, z=0.0). yaw 90.02206420898438
Waypoint 11: (x=-2.033698797225952, y=19.561538696289062, z=0.0). yaw 90.02206420898438


In [150]:
left_lane_points[0].location

AttributeError: 'Waypoint' object has no attribute 'location'

You even got this far... Amazing :)

In [138]:
# Delete all actors
# actor_list.append(vehicle)
for actor in actor_list:
    actor.destroy()

actor_list = []